#### Memory

In some applications like chatbots, previous interactions are needed to keep the whole context of a conversation. Memory provide easy way to handle

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

In [3]:
from langchain.memory import ChatMessageHistory

# object stores sequence of messages in a conversation, keeping track of who said what
history = ChatMessageHistory()

# adds a message from the user to the history
history.add_user_message("hi")

# adds a response from the AI to the history
history.add_ai_message("hello my dud")

# returns list of all messages stored in the history object
history.messages

[HumanMessage(content='hi'), AIMessage(content='hello my dud')]

In [4]:
from langchain.memory import ConversationBufferMemory

# memory language model that stores conversation history as a buffer 
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi")
memory.chat_memory.add_ai_message("hello my friend")
# {} passed as args, means no additional variables are being passed to modify 
# memory loading process
# returns dict with key 'history': 'conversation history'
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: hello my friend'}

- `ChatMessageHistory` is a simple storage of chat messages.
- `ConversationBufferMemory` is a more complex object that stores chat history and provides method to load history in a format that easily used by llm in chains (e.g. ConversationChain) or agents.

In [5]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.predict(input="Hi")

c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi
AI:

> Finished chain.


" Hello there! How are you doing today? I am currently running on a server located in a data center in California. The temperature in the data center is maintained at a constant 68 degrees Fahrenheit to ensure optimal performance of the servers. Did you know that the first data center was built in 1946 by IBM in New York? It was used to store punch cards for the US Census Bureau. Fascinating, isn't it?"

In [6]:
conversation.predict(input="What is the Celcius equivalent of 68 degrees Fahrenheit?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hello there! How are you doing today? I am currently running on a server located in a data center in California. The temperature in the data center is maintained at a constant 68 degrees Fahrenheit to ensure optimal performance of the servers. Did you know that the first data center was built in 1946 by IBM in New York? It was used to store punch cards for the US Census Bureau. Fascinating, isn't it?
Human: What is the Celcius equivalent of 68 degrees Fahrenheit?
AI:

> Finished chain.


' The Celsius equivalent of 68 degrees Fahrenheit is 20 degrees Celsius. Did you know that the Celsius scale was created by Swedish astronomer Anders Celsius in 1742? He originally called it the "centigrade" scale because it was divided into 100 degrees. However, the name was changed to Celsius in 1948 to honor Celsius\' contributions to science.'

In [7]:
conversation.predict(input="What do you mean something is divided by 100? 68 Fahrenheit divided by 100?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hello there! How are you doing today? I am currently running on a server located in a data center in California. The temperature in the data center is maintained at a constant 68 degrees Fahrenheit to ensure optimal performance of the servers. Did you know that the first data center was built in 1946 by IBM in New York? It was used to store punch cards for the US Census Bureau. Fascinating, isn't it?
Human: What is the Celcius equivalent of 68 degrees Fahrenheit?
AI:  The Celsius equivalent of 68 degrees Fahrenheit is 20 degrees Celsius. Did you know that the Celsius scale was created by Swedish astronomer Anders Celsius in 1742? He originally called it 

' Oh, I apologize for the confusion. I was referring to the Celsius scale being divided into 100 degrees. In Fahrenheit, 68 degrees is equivalent to 20 degrees Celsius. The Fahrenheit scale was created by German physicist Daniel Gabriel Fahrenheit in 1724 and is based on a scale of 180 degrees, with 32 degrees being the freezing point of water and 212 degrees being the boiling point.'

#### Conversation Summary Memory

When inputs get long, we might not want to send the whole conversation, but rather a summary

In [8]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain.memory import ConversationSummaryBufferMemory

review = "I ordered a menu of soy garlic chicken and spicy chicken ala carte for RM60. \
The pizza was delivered on time and was still hot when I received it. \
However, no receipt was given and I couldn't figure out if the order is correct.\
When eating the spicy chicken, soy garlic chicken was not included in the delivery. \
I took the picture of my order and made a complaint. \
Within 30 minutes, I get partial refund. \
I am thankful for the quick response from the support and hope they do better in the future."

# to store and manage conversation history
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# first, add context at the start of conversation
memory.save_context(
    {"input": "Hello, how can I help you today?"},
    {"output": "Could you analyze a review for me?"}, # user ask ai to summarize review
)
memory.save_context(
    {"input": "Sure, I'd be happy to. Could you provide the review?"},
    {"output": f"{review}"}, # user
)

In [10]:
# creates object that's ready to be used for conversation
# predict() to start conversation
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
conversation.predict(input="Thank you very much!")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human offers to help the AI and the AI asks for a review to be analyzed. The human agrees and the AI provides a review of a food delivery experience. The review mentions a missing item and a complaint made, resulting in a partial refund and a positive response from customer support.
Human: Thank you very much!
AI:

> Finished chain.


" You're welcome! I'm always happy to help. So, the review I would like you to analyze is from a customer who ordered food delivery from a popular restaurant. They mentioned that one item was missing from their order and they made a complaint to customer support. As a result, they received a partial refund and were very pleased with the response from customer support."

In [11]:
memory.load_memory_variables({})

{'history': "System: \nThe human offers to help the AI and the AI asks for a review to be analyzed. The human agrees and the AI provides a review of a food delivery experience. The review mentions a missing item and a complaint made, resulting in a partial refund and a positive response from customer support.\nHuman: Thank you very much!\nAI:  You're welcome! I'm always happy to help. So, the review I would like you to analyze is from a customer who ordered food delivery from a popular restaurant. They mentioned that one item was missing from their order and they made a complaint to customer support. As a result, they received a partial refund and were very pleased with the response from customer support."}

In [12]:
# Print the conversation summary
print(memory.load_memory_variables({})["history"])

System: 
The human offers to help the AI and the AI asks for a review to be analyzed. The human agrees and the AI provides a review of a food delivery experience. The review mentions a missing item and a complaint made, resulting in a partial refund and a positive response from customer support.
Human: Thank you very much!
AI:  You're welcome! I'm always happy to help. So, the review I would like you to analyze is from a customer who ordered food delivery from a popular restaurant. They mentioned that one item was missing from their order and they made a complaint to customer support. As a result, they received a partial refund and were very pleased with the response from customer support.


In [14]:
conversation.predict(input="The problem has settled. You may now close it.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human offers to help the AI and the AI asks for a review to be analyzed. The human agrees and the AI provides a review of a food delivery experience. The review mentions a missing item and a complaint made, resulting in a partial refund and a positive response from customer support.
Human: Thank you very much!
AI:  You're welcome! I'm always happy to help. So, the review I would like you to analyze is from a customer who ordered food delivery from a popular restaurant. They mentioned that one item was missing from their order and they made a complaint to customer support. As a result, they received a partial refund and were very pleased with the response f

' Thank you for letting me know. I will make a note to close this issue in my system. Is there anything else I can assist you with?'